In [1]:
import os 


In [2]:
%pwd

'/Users/arihantsingla/Documents/CDD_Project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/arihantsingla/Documents/CDD_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir : Path
    base_model_path : Path
    updated_base_model_path : Path
    params_image_size : list
    params_weights : str
    params_classes : int
    params_learning_rate : float
    params_include_top : bool

 

In [6]:
from src.chicken_disease_classification.constants import *
from src.chicken_disease_classification.utils.common import read_yaml, create_directories

In [11]:
class configManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        params = self.params.prepare_base_model
        print("we have created the folder")
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=Path(params.IMAGE_SIZE),    
            params_weights=Path(params.WEIGHTS),
            params_classes=Path(params.CLASSES),
            params_learning_rate=Path(params.LEARNING_RATE),
            params_include_top=Path(params.INCLUDE_TOP),
        )
        return prepare_base_model_config
        

In [12]:
import os 
import zipfile
import urllib.request as request
from src.chicken_disease_classification.utils.common import get_size
from src.chicken_disease_classification import logger
import tensorflow as tf
from zipfile import ZipFile

In [13]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top,
        )
        self.save_model(path=self.config.base_model_path, model=self.model)

    # The @staticmethod decorator indicates that the method does not depend on instance-specific data.
    # It can be called on the class itself without creating an instance. 
    # This is useful for utility functions that perform a task in isolation.
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(units=classes, activation="softmax")(flatten_in)

        full_model = tf.keras.models.Model(inputs=model.input, outputs=prediction)


        ##you can chose the optimizer and loss function as per your need
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self.prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [14]:
try:
    config = configManager()
    prepare_base_model_config = config.prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2025-03-21 00:09:20,864: INFO: common: Reading YAML file from: config/config.yaml]
[2025-03-21 00:09:20,869: INFO: common: YAML file read successfully.]
[2025-03-21 00:09:20,869: INFO: common: Reading YAML file from: params.yaml]
[2025-03-21 00:09:20,871: INFO: common: YAML file read successfully.]
[2025-03-21 00:09:20,871: INFO: common: created directory at: artifacts]


BoxKeyError: "'ConfigBox' object has no attribute 'prepare_base_model'"